In [12]:
import csv
import os
import pandas as pd
import requests

In [13]:
def prepare_multiquery_dict(List_names):
    """
    Prepara un diccionario cuyas llaves son los nombres de las listas de IDs, y cuyos valores son las listas de IDs de elementos,
    como en el siguiente ejemplo: multiquery_dict = { queryname1:["ID11","ID12",...], queryname2:["ID21","ID22",...], ... }. 
    Recibe una lista con los nombres de las listas de IDs (las cuales tienen un ID en cada línea, la primera línea con un 
    identificador de lista tipo FASTA).
    
    Parámetros:
    -----------
        List_names ([string]): Lista de nombres de las listas con IDs.
        
    Salida:
    -------
        multiquery_dict ({string:[string]}): Diccionario que relaciona nombre de listas con las listas de IDs.
    
    """
    multiquery_dict = {}
    for list_name in List_names:
        row_count = 0
        with open(list_name + '.txt') as actual_list:
            csv_reader = csv.reader(actual_list, delimiter='\t')
            for row in csv_reader:
                if row_count == 0: #Primera línea, identificador de lista
                    multiquery_dict[list_name] = []
                    row_count+=1
                    print(row)
                else:
                    multiquery_dict[list_name].append(row[0])
                    row_count+=1
    return multiquery_dict

In [14]:
List_names = ["Lind_up","Lind_down","He_up","He_down","Car(Dmel)_up","Car(Dmel)_down","Bai_F_abd_up","Bai_F_abd_down","Bai_F_head_up","Bai_F_head_down","Bai_M_abd_up","Bai_M_abd_down","Bai_M_head_up","Bai_M_head_down"]
multiquery_dict = prepare_multiquery_dict(List_names)

['>Lind_up']
['>Lind_down']
['>He_up']
['>He_down']
['>Car(Dmel)_up']
['>Car(Dmel)_down']
['>Bai_F_abd_up']
['>Bai_F_abd_down']
['>Bai_F_head_up']
['>Bai_F_head_down']
['>Bai_M_abd_up']
['>Bai_M_abd_down']
['>Bai_M_head_up']
['>Bai_M_head_down']


In [15]:
def multiquery(multiquery_dict,organism,ordered,significance_threshold_method,user_threshold):
    """
    Realiza análisis de enriquecimiento de términos funcionales sobre múltiples listas de elementos D.E., mediante una
    consulta en g:GOSt vía la API de g:Profiler.
    
    Parámetros:
    -----------
        multiquery_dict ({string:[string]}): Diccionario que relaciona nombre de listas con las listas de IDs.
        organism (string): Identificador del organismo de referencia (por ejemplo, 'dmelanogaster' para D. melanogaster)
        ordered (boolean): Indica si las consultas se tratan como listas ordenadas.
        significance_threshold_method (string): Método de corrección por testeo múltiple usado sobre los p-values (por ejemplo, 'fdr' para método Bonferroni-Hochberg FDR)
        user_threshold (float): Valor de corte para los p-value ajustados.
    
    Salida:
    -------
        r.json()['result'] (List): Resultado del análisis en g:GOSt para todas las listas.
    
    """
    r = requests.post(
        url='https://biit.cs.ut.ee/gprofiler/api/gost/profile/',
        json={
        'organism':organism,
        'query':multiquery_dict,
        'combined':True,
        'sources' :[], #All terms.
        'ordered':ordered,
        'user_threshold':user_threshold,
        'significance_threshold_method':significance_threshold_method, #Usar corrección benjamini-hochberg FDR en lugar del default 'g_SCS'.
        'domain_scope':'annotated',#Usar genes anotados en D.mel como conjunto de genes de fondo
        },
        headers={
        'User-Agent':'FullPythonRequest'
        }
    )
    multiquery_result = r.json()['result']
    return multiquery_result

In [16]:
multiquery_result = multiquery(multiquery_dict,'dmelanogaster',True,'fdr',0.20)

In [17]:
def UpSetFile_from_multiqueryResult(List_names,multiquery_result,significance_threshold,output_filename):
    """
    Importante: List_names debe ser la misma lista de nombres de listas que se usó en el multiquery
    
    Parámetros:
    -----------
    
    Salida:
    -------
    
    """
    #Preparación del encabezado
    header = ['term_id','term_name','term_source','term_size']
    
    for list_name in List_names:
        header.append(list_name)
        
    for list_name in List_names:
        header.append(list_name + '_adjpv')
        header.append(list_name + '_int_size')
        
    #Crear archivo    
    with open(output_filename,'w') as f:
        csv_writer = csv.writer(f,delimiter='\t')
        csv_writer.writerow(header)
    
        for term_dict in multiquery_result:
        
            term_id = term_dict['native']
            term_name = term_dict['name']
            term_source = term_dict['source']
            term_size = term_dict['term_size']
            new_row = [term_id,term_name,term_source,term_size]
            p_values = term_dict['p_values']
            intersection_sizes = term_dict['intersection_sizes']
        
            for adjpv in p_values:
                if adjpv<=significance_threshold:
                    new_row.append(1)
                else:
                    new_row.append(0)
            
            for i in range(0,len(p_values)):
                new_row.append(p_values[i])
                new_row.append(intersection_sizes[i])
                
            csv_writer.writerow(new_row)

In [19]:
UpSetFile_from_multiqueryResult(List_names,multiquery_result,0.20,'UpSetFile.txt')